# Plant-wide Simulation: Solving a Reactor-Separator-Recycle Loop

**Objective:** This lesson demonstrates how to simulate a complete chemical process, moving beyond single unit operations. We will model a plant consisting of a reactor and a separator, connected by a critical **recycle stream**. We will implement an iterative algorithm to solve this loop and analyze the overall system performance.

**Learning Goals:**
1.  Understand the concept of a process flowsheet and system-level thinking.
2.  Recognize why recycle streams introduce computational complexity and require an iterative solution.
3.  Implement a simple Plug Flow Reactor (PFR) model and a component separator model.
4.  Write a "successive substitution" algorithm to converge the recycle loop.
5.  Analyze how separator performance and reactor conversion impact the overall process and the size of the recycle stream.

## The Process: Production of Styrene

We will model a simplified process for the dehydrogenation of Ethylbenzene (EB) to produce Styrene (ST) and Hydrogen (H2).
$$ \text{Ethylbenzene (EB)} \rightarrow \text{Styrene (ST)} + \text{Hydrogen (H}_2\text{)} $$

Our plant flowsheet consists of three main parts:
1.  **Mixer:** Fresh feed of pure EB is mixed with a recycle stream containing unreacted EB.
2.  **PFR Reactor:** The combined feed enters the reactor where some of the EB is converted to ST and H2.
3.  **Separator:** The reactor outlet is cooled and sent to a separator. We assume this unit perfectly separates the H2 as a gas product. It then separates a fraction of the ST as the main product, while the remaining ST and all unreacted EB form the recycle stream.

![Flowsheet Diagram](https://www.researchgate.net/publication/260317958/figure/fig2/AS:392379984039940@1470562086497/Flowsheet-of-the-styrene-production-process-Taken-from-4.png)

*(Image MAY not be 100% accurate depending on what link actually works on this thing)*

## The Challenge: Solving the Recycle Loop

We cannot solve this flowsheet sequentially. The input to the Mixer (and thus the Reactor) depends on the Recycle stream, but the Recycle stream depends on the output of the Separator, which in turn depends on the output of the Reactor. This creates a circular dependency.

**The Solution: Iteration**
We must use an iterative approach called **successive substitution**:
1.  **Guess** the initial properties of the recycle stream (flow rates of each component).
2.  Use this guess to calculate the Mixer and then the Reactor.
3.  Calculate the Separator based on the reactor outlet.
4.  This gives us a **new, calculated** recycle stream.
5.  **Compare** the new recycle stream to our initial guess. If they are close enough (converged), we are done. If not, we use the newly calculated stream as our **next guess** and repeat from step 2.

This is analogous to "tearing" the recycle stream in a professional process simulator.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# --- Define Global Parameters ---
# Component indices: 0=EB, 1=ST, 2=H2

# Fresh Feed
F_feed = np.array([100.0, 0.0, 0.0]) # Molar flow (kmol/hr)

# Reactor Parameters
k = 0.35 # First-order rate constant (1/hr)
V_reactor = 300 # Reactor Volume (m^3)

print("Parameters and libraries are ready.")

## Part 1: Unit Operation Models

First, we need to create Python functions to model our two main unit operations.

In [ ]:
def pfr_reactor(F_in):
    """Models a simple PFR for a first-order reaction EB -> ST + H2."""
    F_total_in = np.sum(F_in)
    # Handle case of zero flow
    if F_total_in < 1e-6:
        return F_in
        
    tau = V_reactor / F_total_in # Assuming constant volumetric flow for simplicity
    
    # Analytical solution for first-order reaction in a PFR
    F_eb_out = F_in[0] * np.exp(-k * tau)
    
    # Calculate amount of EB reacted
    eb_reacted = F_in[0] - F_eb_out
    
    # Calculate outlet flows of products
    F_st_out = F_in[1] + eb_reacted
    F_h2_out = F_in[2] + eb_reacted
    
    return np.array([F_eb_out, F_st_out, F_h2_out])

def separator(F_in, styrene_split_fraction):
    """Models a separator.
       - All H2 leaves in the gas product stream.
       - A specified fraction of Styrene leaves in the main product stream.
       - All EB and the rest of the Styrene leave in the recycle stream.
    """
    # Gas Product (pure H2)
    F_gas_prod = np.array([0.0, 0.0, F_in[2]])
    
    # Main Product (a fraction of the Styrene)
    styrene_to_product = F_in[1] * styrene_split_fraction
    F_main_prod = np.array([0.0, styrene_to_product, 0.0])
    
    # Recycle Stream (all EB and remaining Styrene)
    styrene_to_recycle = F_in[1] * (1 - styrene_split_fraction)
    F_recycle = np.array([F_in[0], styrene_to_recycle, 0.0])
    
    return F_gas_prod, F_main_prod, F_recycle

print("Unit operation models defined.")

## Part 2: The Interactive Flowsheet Simulation

Now we will create a master function that takes our key design parameters and solves the entire flowsheet iteratively. We will then wrap this function in an interactive widget.

In [ ]:
def solve_flowsheet(styrene_split, single_pass_conversion, plot=True):
    """Iteratively solves the entire plant flowsheet."""
    
    # --- Update reactor parameter k based on desired single-pass conversion ---
    # X = 1 - exp(-k*tau) => k = -ln(1-X)/tau
    # Use fresh feed flow to define a reference tau
    ref_tau = V_reactor / np.sum(F_feed)
    k_local = -np.log(1 - single_pass_conversion) / ref_tau
    
    def pfr_reactor_local(F_in):
        # A local version of the reactor model using the updated k
        F_total_in = np.sum(F_in)
        if F_total_in < 1e-6: return F_in
        tau = V_reactor / F_total_in
        F_eb_out = F_in[0] * np.exp(-k_local * tau)
        eb_reacted = F_in[0] - F_eb_out
        F_st_out = F_in[1] + eb_reacted
        F_h2_out = F_in[2] + eb_reacted
        return np.array([F_eb_out, F_st_out, F_h2_out])

    # --- Iteration Setup ---
    F_recycle_guess = np.array([0.0, 0.0, 0.0]) # Start with a guess of zero recycle
    tolerance = 1e-6
    max_iterations = 100
    
    for i in range(max_iterations):
        # 1. Mixer
        F_reactor_in = F_feed + F_recycle_guess
        
        # 2. Reactor
        F_reactor_out = pfr_reactor_local(F_reactor_in)
        
        # 3. Separator
        _, _, F_recycle_calc = separator(F_reactor_out, styrene_split)
        
        # 4. Check for Convergence
        error = np.sum(np.abs(F_recycle_calc - F_recycle_guess))
        if error < tolerance:
            # CONVERGED!
            # Recalculate final stream values with converged recycle flow
            F_reactor_in_final = F_feed + F_recycle_calc
            F_reactor_out_final = pfr_reactor_local(F_reactor_in_final)
            F_gas_prod, F_main_prod, F_recycle_final = separator(F_reactor_out_final, styrene_split)
            
            # Plotting and printing results
            if plot:
                plot_results(F_feed, F_recycle_final, F_reactor_in_final, F_main_prod, F_gas_prod, i+1)
            return
        
        # 5. Not converged, update guess for next iteration
        # Use damping to improve stability
        F_recycle_guess = 0.5 * F_recycle_guess + 0.5 * F_recycle_calc
    
    print("Flowsheet did not converge!")

def plot_results(F_feed, F_recycle, F_reactor_in, F_main_prod, F_gas_prod, iterations):
    """Helper function to visualize the results."""
    
    streams = ['Fresh Feed', 'Recycle', 'Reactor Feed', 'Styrene Prod', 'H2 Prod']
    data = {
        'Ethylbenzene': [F_feed[0], F_recycle[0], F_reactor_in[0], F_main_prod[0], F_gas_prod[0]],
        'Styrene': [F_feed[1], F_recycle[1], F_reactor_in[1], F_main_prod[1], F_gas_prod[1]],
    }

    fig, ax = plt.subplots(figsize=(12, 7))
    width = 0.35
    bottom = np.zeros(len(streams))
    
    for component, component_flows in data.items():
        p = ax.bar(streams, component_flows, width, label=component, bottom=bottom)
        bottom += component_flows

    ax.set_ylabel('Molar Flow (kmol/hr)', fontsize=12)
    ax.set_title(f'Converged Flowsheet Stream Compositions (in {iterations} iterations)', fontsize=16, weight='bold')
    ax.legend(loc='upper right')
    plt.show()

    recycle_ratio = np.sum(F_recycle) / np.sum(F_feed)
    print(f"Recycle Ratio (Recycle/Feed): {recycle_ratio:.2f}")
    overall_conversion = (F_feed[0] - F_recycle[0]) / F_feed[0] * 100
    print(f"Overall Conversion of Ethylbenzene: {overall_conversion:.1f}%")

print("Flowsheet solver function defined.")

### Interactive Dashboard
Use the sliders below to change key design parameters and see how they affect the entire process. 
*   **Single-Pass Conversion:** How much EB is converted in one trip through the reactor? A lower conversion requires a more powerful reactor (or lower throughput).
*   **Styrene Split Fraction:** How good is our separator? This is the fraction of styrene that goes to the final product. A value of 1.0 would mean perfect separation of ST from EB.

In [ ]:
# Create interactive widgets
style = {'description_width': 'initial'}
conversion_slider = widgets.FloatSlider(value=0.6, min=0.1, max=0.9, step=0.05, description='Reactor Single-Pass Conversion:', style=style)
split_slider = widgets.FloatSlider(value=0.9, min=0.5, max=1.0, step=0.05, description='Separator Styrene Split to Product:', style=style)

# Link the widgets to the simulation function
widgets.interactive(solve_flowsheet, styrene_split=split_slider, single_pass_conversion=conversion_slider, plot=widgets.fixed(True))

## Student Challenges and Analysis

Use the interactive dashboard to answer these critical process design questions:

1.  **The "Snowball" Effect of Recycle:** Set the `Reactor Single-Pass Conversion` to a low value, like 0.2 (20%). What happens to the size of the Recycle stream and the Reactor Feed stream? Why is a low single-pass conversion often undesirable in a plant with a recycle loop?

2.  **The Importance of Separation:** Set the `Reactor Single-Pass Conversion` back to 0.6. Now, decrease the `Separator Styrene Split` to a low value like 0.5. This represents a poor separation where a lot of valuable product is being sent back to the reactor. What happens to the overall process? What does this tell you about the importance of the separation unit in the profitability of a plant?

3.  **Overall vs. Single-Pass Conversion:** Notice that the **Overall Conversion** is always very high (approaching 100%). Why is this the case, even when the single-pass conversion is low? (Hint: What is the only way for unreacted EB to leave the system in our model?)